<a href="https://colab.research.google.com/github/FahadEbrahim/AdaptIRC/blob/main/NLBSE2024_AdaptIRC_Colab_Submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to run

AdaptIRC:


* This notebook implements the adapter approach (AdaptIRC) on the NLBSE 2024 Issue Report Classification task.

* To run the notebook in Colab, just change the environment to GPU through: Runtime >> Change runtime type >> Hardware Accelerator >> GPU.

* You may require WANDB token if using newer versions of transformers lib

# Install Dependencies

The code uses the adapters library, Huggingface datasets and accelerate. The adapters would install the transformers libraries
For reproducibility, the versions have been specified.


In [1]:
!pip install adapters==0.0.0.dev20231116
!pip install datasets==2.14.7
!pip install accelerate==0.24.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00


# Get the Dataset from Github

The code clones the Github repository provided by NLBSE'2024 (https://github.com/nlbse2024/issue-report-classification):



In [2]:
!git clone https://github.com/nlbse2024/issue-report-classification.git

Cloning into 'issue-report-classification'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 76 (delta 37), reused 39 (delta 14), pack-reused 0
Receiving objects: 100% (76/76), 1.59 MiB | 7.86 MiB/s, done.
Resolving deltas: 100% (37/37), done.


# Import Libraires

Here, we are importing libraries that would be used throughout the notebook. (Pandas, Json, OS, Sklearn, numpy, collections, transformers, adapters, random, torch, re [regular expression] ).

In [3]:
import pandas as pd
import json
import os
from collections import defaultdict
import numpy as np
import torch
import random
import re

from transformers import TextClassificationPipeline,AutoConfig
from transformers import TrainingArguments, EvalPrediction
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoConfig
from transformers import TrainingArguments, EvalPrediction
from transformers import set_seed
from transformers import EarlyStoppingCallback, IntervalStrategy,TrainerCallback

from adapters import AdapterTrainer,AutoAdapterModel
from datasets import Dataset
from sklearn.metrics import classification_report, accuracy_score,recall_score,f1_score,precision_score

# Setting Seed

These lines set the seed for reproducability for several libraries ( torch, random, numpy, transformers)

In [4]:
RANDOM_SEED = 42
set_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [5]:
import sys
print(sys.modules.keys())

dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', 'types', 'enum', '_sre', 'sre_constants', 'sre_parse', 'sre_compile', '_collections_abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', '_functools', 'functools', '_locale', 'copyreg', 're', 'warnings', '_stat', 'stat', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_distutils_hack', 'importlib._bootstrap', 'importlib._bootstrap_external', 'importlib', 'importlib._abc', 'contextlib', 'importlib.util', 'importlib.machinery', 'google', 'google.ai', 'google.cloud', 'google.logging', 'google.iam', 'mpl_toolkits', 'sphinxcontrib', 'sitecustomize', 'site', 'runpy', 'ipykernel._version', '_json', 'json.scanner', 'json.decoder', 'json.encoder', 'js

# Dataset

Reading the dataset through NLBSE Github repository:

In [6]:
train_set = pd.read_csv("/content/issue-report-classification/data/issues_train.csv")
test_set = pd.read_csv("/content/issue-report-classification/data/issues_test.csv")

# Dataset Processing

In [7]:
# There were some Nan values that causes some issues, so, they are replaced with a single space
train_set=train_set.fillna(' ')
test_set=test_set.fillna(' ')

This function is used to pre-process the issues with various steps (
  
  * removing strings between triple Quotes
  * Remove new lines
  * Remove Links
  * Remove digits
  * Remove special characters except the question mark
  * Remove multiple spaces


In [8]:
def preprocess(issues):
    processed_issues = []

    for issue in issues:

        # Remove strings between triple quotes
        issue = re.sub(r'```.*?```', ' ', issue, flags=re.DOTALL)

        # Remove new lines
        issue = re.sub(r'\n', ' ', issue)

        # Remove links
        issue = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', issue)

        # Remove digits
        issue = re.sub(r'\d+', ' ', issue)

        # Remove special characters except the question marks
        issue = re.sub(r'[^a-zA-Z0-9?\s]', ' ', issue)
        issue = re.sub(r'\s+', ' ', issue)

        processed_issues.append(issue)

    return processed_issues

In [9]:
# Apply the pre-process function for both train and testing sets on both the title and body.
train_set['title'] = preprocess(train_set['title'])
train_set['body'] = preprocess(train_set['body'])

test_set['title'] = preprocess(train_set['title'])
test_set['body'] = preprocess(train_set['body'])

In [10]:
# This code is taken from NLBSE
# creating the dataset with grouping it via repositry (repo)

repos = list(set(train_set["repo"].unique()))

train_set.groupby(["repo", "label"]).size().unstack(fill_value=0)

# Combining the title and body for a new field called text.
def process_dataset(dataset):
    dataset['text'] = dataset['title'] + " " + str(dataset['body'])
    dataset = dataset[['text', 'label', 'repo']]
    return dataset

train_set = process_dataset(train_set)
test_set = process_dataset(test_set)

group_by_repo = lambda dataset: {
    repo: Dataset.from_pandas(dataset[dataset["repo"] == repo]).class_encode_column("label")
    for repo in dataset["repo"].unique()
}

train_sets = group_by_repo(train_set)
test_sets = group_by_repo(test_set)

datasets = {
    repo: {'train': train_sets[repo], 'test': test_sets[repo]} for repo in train_sets.keys()
}

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/300 [00:00<?, ? examples/s]

# Model Configuration

Here is the new important code: Setting the configurations of the adapters and transformer model.

In [11]:
# Model name to be used is: Roberta-Base
model_name = "roberta-base"

# The tokenizer is based on Roberta. The configurations are: Max_length = 256, truncation = true, padding = max_length.
tokenizer = AutoTokenizer.from_pretrained(model_name,max_length=256,truncation=True, padding="max_length")

# Configuration: We have 3 labels: Bug, Enhancment, Question.
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=3,
)

# Configuration of the Adapter model.
model = AutoAdapterModel.from_pretrained(
    model_name,
    config=config,
)

# Metrics used for evaluation (accuracy, precision, recall and F1)
def compute_accuracy(p: EvalPrediction):
  labels = p.label_ids
  preds = np.argmax(p.predictions, axis=1)
  accuracy = accuracy_score(y_true=labels, y_pred=preds)
  recall = recall_score(y_true=labels, y_pred=preds,average="weighted")
  precision = precision_score(y_true=labels, y_pred=preds,average="weighted")
  f1 = f1_score(y_true=labels, y_pred=preds,average="weighted")
  return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# The function used to tokenize the issues. The same settings
# Max_length = 256, truncation = true, padding = max_length.

def encode_batch(batch):
  return tokenizer(batch["text"], max_length=256, truncation=True, padding="max_length")

# Having a data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Creating Training and Infering Adapters

In [30]:
# Clearing the GPU just in case
import torch
torch.cuda.empty_cache()

The training occurs at every repository:
* The train set is divided into: train and validate with the percentage of 30%.
* A classfication head is attached to the model defining the number of labels to be 3 and defining the labels.
* Initilaising the training of the Adapter
* Using Adapter Droput Trainer as the Callback.
* Configuring the adapter configuarion.
* Configure the trainer
* Adding the callback.
* Start training the adapter
* Evalauting the adapter

In [38]:
results = defaultdict(dict)

for repo in datasets.keys():

    # Extracting the training and testing sets from the dataset per repo
    train_set = datasets[repo]['train']
    test_set = datasets[repo]['test']

    # Tokenizing the training set
    train_set = train_set.map(encode_batch, batched=True)
    train_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    # Creating a Validation set:
    train_val_set = train_set.train_test_split(test_size=0.3,seed = RANDOM_SEED)
    train_set = train_val_set['train']
    val_set = train_val_set['test']

    # Adapter Name and Saving Directory
    adapter_name = "Report_" + repo
    adapter_name = adapter_name.replace('/','_')
    saved_directoy = "./training_output/" + adapter_name + ""

    # Add an Adapter
    model.add_adapter(adapter_name,overwrite_ok=True)
    # Add a matching classification head

    # Add a Classification Head
    model.add_classification_head(
    adapter_name,
    num_labels=3,
    id2label={ 0: "bug", 1: "feature",2:"question"},overwrite_ok=True)

    # Initilaize the adapter training
    model.train_adapter(adapter_name)

    # Create an Adapter Callback
    class AdapterDropTrainerCallback(TrainerCallback):
        def on_step_begin(self, args, state, control, **kwargs):
          skip_layers = list(range(np.random.randint(0, 11)))
          kwargs['model'].set_active_adapters(adapter_name, skip_layers=skip_layers)

        def on_evaluate(self, args, state, control, **kwargs):
          kwargs['model'].set_active_adapters(adapter_name, skip_layers=None)

    # Configure Training arguements
    training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=200,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir=saved_directoy,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    seed= RANDOM_SEED
    )

    # Configure the Adapter Trainer
    trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_accuracy,
    data_collator=data_collator,)

    # Add the callback to the trainer
    trainer.add_callback(AdapterDropTrainerCallback())

    # Start training the adapter
    trainer.train()

    # Evaluating the Adapter and printing the results.
    eval_results = trainer.evaluate()
    print(eval_results)

    # Merging the Repo
    model.merge_adapter(adapter_name)

    # This part is for inferencing
    # A TextClassificationPipeline is being created with the same settings
    # max_length = 256, padding = 'max_length',truncation =True

    classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer,
                                            device=training_args.device.index,
                                            max_length = 256, padding = 'max_length',truncation =True)
    y_pred = []

    # Looping through the testing set and getting the prediction labels.
    for i in range(len(test_set['text'])):
        text = test_set['text'][i]
        y = classifier(text)
        y = y[0]['label']
        label_id = model.config.label2id[y]
        y_pred.append(label_id)

    # Calcualting and Adding the metrics
    results[repo]['metrics'] = classification_report(test_set['label'], y_pred, digits=4, output_dict=True)
    results[repo]['predictions'] = y_pred
    results['label_mapping'] = {train_set.features["label"].int2str(x): x for x in range(train_set.features["label"].num_classes)}

    # Clearing the adapter and CH for every repo.
    model.delete_adapter(adapter_name)
    model.delete_head(adapter_name)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,0.760700
400,0.286000
600,0.129100
800,0.072800
1000,0.057000
1200,0.052700
1400,0.050400


The model 'RobertaAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 

{'eval_loss': 1.993656039237976, 'eval_accuracy': 0.7777777777777778, 'eval_precision': 0.7877836566725456, 'eval_recall': 0.7777777777777778, 'eval_f1': 0.775658153077508, 'eval_runtime': 1.3576, 'eval_samples_per_second': 66.291, 'eval_steps_per_second': 2.21, 'epoch': 200.0}


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,1.090200
400,0.474200
600,0.143500
800,0.086500
1000,0.074600
1200,0.062300
1400,0.060000


The model 'RobertaAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 

{'eval_loss': 3.3290584087371826, 'eval_accuracy': 0.6666666666666666, 'eval_precision': 0.6721200980392157, 'eval_recall': 0.6666666666666666, 'eval_f1': 0.6683352468427095, 'eval_runtime': 1.3537, 'eval_samples_per_second': 66.486, 'eval_steps_per_second': 2.216, 'epoch': 200.0}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


# Metrics

This has been taken from the NLBSE repo, so, they are not commented.

In [39]:
print(results['label_mapping'])
for repo in repos:
    print(repo)
    print(json.dumps(results[repo]['metrics'], indent=4))

{'bug': 0, 'feature': 1, 'question': 2}
microsoft/vscode
{
    "0": {
        "precision": 0.8910891089108911,
        "recall": 0.9,
        "f1-score": 0.8955223880597015,
        "support": 100
    },
    "1": {
        "precision": 0.900990099009901,
        "recall": 0.91,
        "f1-score": 0.9054726368159204,
        "support": 100
    },
    "2": {
        "precision": 0.9591836734693877,
        "recall": 0.94,
        "f1-score": 0.9494949494949495,
        "support": 100
    },
    "accuracy": 0.9166666666666666,
    "macro avg": {
        "precision": 0.91708762713006,
        "recall": 0.9166666666666666,
        "f1-score": 0.9168299914568571,
        "support": 300
    },
    "weighted avg": {
        "precision": 0.9170876271300601,
        "recall": 0.9166666666666666,
        "f1-score": 0.916829991456857,
        "support": 300
    }
}
tensorflow/tensorflow
{
    "0": {
        "precision": 0.9204545454545454,
        "recall": 0.81,
        "f1-score": 0.8617021276

In [40]:
class_metrics_sum = defaultdict(defaultdict)
labels = [key for key in results[repos[0]]['metrics'].keys() if key.isnumeric()]

for repo in repos:
    for label in labels:
        for metric in results[repo]['metrics'][label]:
            class_metrics_sum[label][metric] = class_metrics_sum[label].get(metric, 0) + results[repo]['metrics'][label][metric]

class_metrics_avg = {
    label: {
        metric: class_metrics_sum[label][metric] / len(repos)
        for metric in class_metrics_sum[label]
    }
    for label in labels
}

# add the average of the metric over all classes
class_metrics_avg['average'] = {
    metric: sum(class_metrics_avg[label][metric] for label in labels)
    / len(labels)
    for metric in class_metrics_avg[labels[0]]
}

# add to the results
results['overall'] = {
    'metrics': class_metrics_avg
}

In [41]:
results['overall']

{'metrics': {'0': {'precision': 0.9010907821551386,
   'recall': 0.8879999999999999,
   'f1-score': 0.8936217738921546,
   'support': 100.0},
  '1': {'precision': 0.8976271184810789,
   'recall': 0.9040000000000001,
   'f1-score': 0.9005929800105748,
   'support': 100.0},
  '2': {'precision': 0.8899241513030333,
   'recall': 0.892,
   'f1-score': 0.8902754752023044,
   'support': 100.0},
  'average': {'precision': 0.8962140173130836,
   'recall': 0.8946666666666667,
   'f1-score': 0.8948300763683447,
   'support': 100.0}}}

In [42]:
#The output json file would be created containing the results.
!mkdir Final_Results
OUTPUT_PATH = "Final_Results"
output_file_name = 'results.json'
with open(os.path.join(OUTPUT_PATH, output_file_name), 'w') as fp:
    json.dump(results, fp)

# References & Ack

This notebook uses codes from:
* https://github.com/adapter-hub/adapters/blob/main/notebooks/01_Adapter_Training.ipynb
* https://github.com/adapter-hub/adapters/blob/main/notebooks/05_Adapter_Drop_Training.ipynb
* https://huggingface.co/docs/transformers/tasks/sequence_classification
* https://github.com/nlbse2024/issue-report-classification/blob/main/2-Template-SetFit.ipynb
